In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML_analysis") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "32g") \
    .getOrCreate()

24/05/14 20:07:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Outline
(1) Used machine learning models to predict the click rates of ad impressions
<br>(2) Used simulation to find the optimal bidding price

### Loading Dataframe

In [2]:
df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/yl9709/combined_training_before_normalize.txt")

In [3]:
df.head()

24/05/14 20:07:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(bidID='b006d2236cd8786f8269e3bd60dd795f', timestamp=20130318000100765, logtype=1, iPinYouID='c43a65560b3c8adc7cfe24ef38e82398', userAgent='Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; Apache; .NET CLR 2.0.50727; 360SE)', IP='60.190.192.*', region=94, city=100, adExchange=2.0, domain='eF1gtp57dNFrw9Kbu-', URL='373ca49d791e5fc3bae3b23ab7b39f8', URLID=None, adSlotID='426993709', width=468, height=60, visibility='0', format='0', floorPrice=13, creativeID='28425676c2469651ba969a88529eeec8', biddingPrice=300, payingPrice=13, keyPageURL='9f4e2f16b6873a7eb504df6f61b24044', advertiserID=0, userTags='0', numClick=0, conversion=0, datetime='2013-03-18 00:01:00.765', weekday='Monday', hour=0)

In [4]:
# Select necessary columns
from pyspark.sql.functions import col

df = df.select(
    col("numClick").cast("int"),
    col("region").cast("int"),
    col("city").cast("int"),
    col("adExchange").cast("int"),
    col("URL"),
    col("width").cast("int"),
    col("height").cast("int"),
    col("floorPrice").cast("int"),
    col("weekday"),
    col("hour").cast("int"),
    col("payingPrice").cast("int")
)

# Check the data
df.show(5)

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+
|       0|    94| 100|         2|373ca49d791e5fc3b...|  468|    60|        13| Monday|   0|         13|
|       0|    40|  42|         2|ba5f614bb0a90cd04...|  728|    90|         5| Monday|   0|         25|
|       0|    40|  45|         1|f4766aa78c9debf5a...|  160|   600|         0| Monday|   0|         13|
|       0|    80|  85|         2|5db0c3f5ddab9fd58...|  300|   250|         5| Monday|   0|         24|
|       0|     0|   0|         2|aa1ab4011d63ca032...|  300|   250|         5| Monday|   0|         57|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+
only showing top 5 rows



Each row represents a bid for an impression

In [5]:
from pyspark.sql.functions import monotonically_increasing_id

# Adding a unique ID to test_df before splitting into train/test or applying transformations
df = df.withColumn("unique_id", monotonically_increasing_id())

### Data Transformation

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# Indexing and encoding categorical columns
categoricalColumns = ['region', 'city', 'adExchange', 'weekday']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

# Assembling vector
assemblerInputs = [c + "classVec" for c in categoricalColumns] + ['width', 'height', 'floorPrice', 'hour']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]


### Train (70%) - Test (30%) Split

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Set up the evaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="numClick", metricName="areaUnderROC")

train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

### (1) Logistic Regression

In [8]:
# Logistic Regression
lr = LogisticRegression(labelCol="numClick", featuresCol="features", maxIter=10)

In [9]:
pipeline_lr = Pipeline(stages=stages + [lr])

In [10]:
model_lr = pipeline_lr.fit(train_df)

24/05/14 20:08:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/05/14 20:08:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [11]:
predictions_lr = model_lr.transform(test_df)
print("Logistic Regression AUC: ", evaluator.evaluate(predictions_lr))

Logistic Regression AUC:  0.5712724408492721


### (2) Gradient Boosting Trees

In [12]:
# Gradient Boosting Trees
gbt = GBTClassifier(labelCol="numClick", featuresCol="features", maxIter=10)

In [13]:
pipeline_gbt = Pipeline(stages=stages + [gbt])

In [14]:
model_gbt = pipeline_gbt.fit(train_df)

In [15]:
predictions_gbt = model_gbt.transform(test_df)
print("Gradient Boosting AUC: ", evaluator.evaluate(predictions_gbt))

Gradient Boosting AUC:  0.6223684451391842


In [16]:
predictions_gbt.printSchema()

root
 |-- numClick: integer (nullable = true)
 |-- region: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- adExchange: integer (nullable = true)
 |-- URL: string (nullable = true)
 |-- width: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- floorPrice: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- payingPrice: integer (nullable = true)
 |-- unique_id: long (nullable = false)
 |-- regionIndex: double (nullable = false)
 |-- regionclassVec: vector (nullable = true)
 |-- cityIndex: double (nullable = false)
 |-- cityclassVec: vector (nullable = true)
 |-- adExchangeIndex: double (nullable = false)
 |-- adExchangeclassVec: vector (nullable = true)
 |-- weekdayIndex: double (nullable = false)
 |-- weekdayclassVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (n

In [17]:
predictions_gbt.show(5)

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-----------+---------------+---------+---------------+---------------+------------------+------------+---------------+--------------------+--------------------+--------------------+----------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|regionIndex| regionclassVec|cityIndex|   cityclassVec|adExchangeIndex|adExchangeclassVec|weekdayIndex|weekdayclassVec|            features|       rawPrediction|         probability|prediction|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-----------+---------------+---------+---------------+---------------+------------------+------------+---------------+--------------------+--------------------+--------------------+----------+
|       0|     0|   0|         1|102286d4be01b0804...|  950|    90|         0| Mon

### (3) Support Vector Machine


In [18]:
# Support Vector Machine
svm = LinearSVC(labelCol="numClick", featuresCol="features", maxIter=10)

In [19]:
pipeline_svm = Pipeline(stages=stages + [svm])

In [20]:
model_svm = pipeline_svm.fit(train_df)

In [21]:
predictions_svm = model_svm.transform(test_df)
print("SVM AUC: ", evaluator.evaluate(predictions_svm))

SVM AUC:  0.5071795590023994


### Add prediction result to test_df

In [22]:
# extract positive probability from GBT model
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType

# Define a UDF to extract the second element from the probability vector
get_positive_prob = udf(lambda v: float(v[1]), FloatType())

# Add a new column to predictions_gbt with the positive class probability
predictions_gbt = predictions_gbt.withColumn("positive_prob", get_positive_prob(col("probability")))

In [23]:
test_df_with_prob = test_df.join(predictions_gbt, "unique_id").select(test_df["*"], predictions_gbt["positive_prob"])

In [24]:
test_df_with_prob.count()

2007563

In [25]:
test_df.count()

2007563

In [26]:
test_df_with_prob.show(30)

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|positive_prob|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+
|       0|   216| 233|         1|177a166119fcf41b8...|  160|   600|         0| Monday|   0|         50|       26|   0.06616997|
|       0|    94|  96|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|         14|      474|  0.066168584|
|       0|   216| 219|         2|14813226981bea2d8...|  300|   250|         5| Monday|   0|         24|     1806|   0.06616997|
|       0|    80|  82|         1|1f1640b91b6c10733...|  300|   250|         0| Monday|   0|         53|     2453|   0.06632049|
|       0|   238| 239|         1|d8df91a3689429534...|  300|   250|         0| Monday|   0|         32| 

In [27]:
test_df_with_prob.printSchema()

root
 |-- numClick: integer (nullable = true)
 |-- region: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- adExchange: integer (nullable = true)
 |-- URL: string (nullable = true)
 |-- width: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- floorPrice: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- payingPrice: integer (nullable = true)
 |-- unique_id: long (nullable = false)
 |-- positive_prob: float (nullable = true)



In [28]:
sum_numClick = test_df.agg(sum("numClick").alias("sum_numClick")).collect()[0]["sum_numClick"]
print(sum_numClick)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

### Finding Optimal Bid Price

### Define the Bidding Function:
(1) Bidding Price = α × Predicted Click Rate * ROI Function
<br>
<br>(2) Simulation Function: Create a function to simulate bidding based on a given α and calculate the total clicks won and the total cost. 
<br>
<br>(3) Implement Binary Search: Use binary search to find the optimal α to maximize click probability while staying under the budget.

In [29]:
# Sum bidding price to determine the budget.
# The max bidding in the simulation will be less than this.

from pyspark.sql.functions import sum
total_paying_price = test_df_with_prob.agg(sum("payingPrice").alias("total_paying_price")).collect()[0]["total_paying_price"]
print("Total Paying Price: ", total_paying_price)

Total Paying Price:  169989899


In [51]:
budget = 170017031/1000
print(budget)

170017.031


In [32]:
from pyspark.sql.functions import avg

average_numClick = test_df_with_prob.agg(avg("numClick").alias("average_numClick")).show()

+--------------------+
|    average_numClick|
+--------------------+
|7.387065810637076E-4|
+--------------------+



In [33]:
# (avg winning price / average click through rate) to determine the range of the alpha

from pyspark.sql.functions import avg, max, min
results = test_df.agg(
    avg("payingPrice").alias("avg_paying_price"),
    avg("numClick").alias("avg_click_rate"),
    max("payingPrice").alias("max_paying_price"),
).collect()[0]

ratio_avg = results["avg_paying_price"] / results["avg_click_rate"]
print("Ratio of average payingPrice to average clickRate:", ratio_avg)
print("max payingPrice:", results["max_paying_price"])

# bid price = click through rate * alpha
# 114643 / 1000

Ratio of average payingPrice to average clickRate: 114625.69049224546
max payingPrice: 300


In [34]:
print(114643 / 1000)

114.643


In [35]:
results["avg_paying_price"]

84.67475192559337

In [36]:
roi_dict = {0: 1.0,
 1: 1.5,
 2: 0.5,
 3: 0.5,
 4: 1.0,
 5: 1.5,
 6: 1.5,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 0.5,
 11: 1.0,
 12: 1.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.5,
 19: 1.0,
 20: 1.5,
 21: 1.5,
 22: 1.5,
 23: 1.5}

In [37]:
# simulate bidding to based on a given alpha - returns the total clicks and total costs
def simulateBidding(alpha, test_df, budget):
    total_spent = 0
    total_clicks = 0
    roi_multiplier = 1

    for row in test_df.collect():
        predicted_click_rate = row.positive_prob
        winning_price = row.payingPrice / 1000.0
        bidding_hour = row.hour
        
        bid_price = alpha * (predicted_click_rate * roi_dict[bidding_hour])
        if bid_price > winning_price:
            total_spent += winning_price
            total_clicks =  total_clicks + row.numClick


    return total_clicks, total_spent


# Use binary search to find the alpha that maximizes the click rate while staying under the budget
def binarySearch(low, high, test_df, budget):
    # returns the alpha
    alpha = 0
    max_clicks = 0
    count = 0

    while high - low > 0.01 and count < 100:
        
        mid = (low + high) / 2
        clicks, spend = simulateBidding(mid, test_df, budget)
        print(mid)
        
        if count % 100 == 0:
            print(mid)
            print(clicks, spend)
            
        if spend > budget:
            high = mid
        else:
            low = mid
            if clicks > max_clicks:
                max_clicks = clicks
                alpha = mid
        
        count += 1

    return alpha

### Find alpha under various scenarios

In [38]:
alpha = binarySearch(0, 1200, test_df_with_prob, 5000)
print(alpha)

600.0
600.0
1483 169989.89900033496
300.0
150.0
75.0
37.5
18.75
9.375
4.6875
2.34375
1.171875
0.5859375
0.29296875
0.439453125
0.3662109375
0.32958984375
0.311279296875
0.3021240234375
0.3021240234375


In [39]:
alpha = binarySearch(0, 1200, test_df_with_prob, 10000)
print(alpha)

600.0
600.0
1483 169989.89900033496
300.0
150.0
75.0
37.5
18.75
9.375
4.6875
2.34375
1.171875
0.5859375
0.29296875
0.439453125
0.5126953125
0.47607421875
0.494384765625
0.4852294921875
0.4852294921875


In [55]:
alpha = binarySearch(0, 1200, test_df_with_prob, 15000)
print(alpha)

600.0
600.0
1483 169989.89900033496


300.0


150.0


75.0


37.5


...

0.732421875


0.6591796875


0.62255859375


0.604248046875


0.6134033203125
0.604248046875


### Find ROI under scenario 1: Budget = 500 CNY


In [59]:
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import DoubleType

# UDF to compute the new column value
def calculate_value(t, alpha, positive_prob):
    return alpha * positive_prob * roi_dict.get(t)

calculate_value_udf = udf(calculate_value, DoubleType())

alpha = 0.3021240234375

test_df_with_prob = test_df_with_prob.withColumn('optimalBid', calculate_value_udf(col('hour'), lit(alpha), col('positive_prob')))

# Show the DataFrame with the new column
test_df_with_prob.show()

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|positive_prob|          optimalBid|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|       0|   216| 233|         1|177a166119fcf41b8...|  160|   600|         0| Monday|   0|         50|       26|   0.06616997|0.019991537487840105|
|       0|    94|  96|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|         14|      474|  0.066168584|0.019991118801954144|
|       0|   216| 219|         2|14813226981bea2d8...|  300|   250|         5| Monday|   0|         24|     1806|   0.06616997|0.019991537487840105|
|       0|    80|  82|         1|1f1640b91b6c10733...|  300|   250|         0| Monday|   0|         53|   

In [60]:
# Filter rows where optimalBid >= payingPrice
filtered_df = test_df_with_prob.filter(col('optimalBid') >= col('payingPrice')/1000 )

# Count the number of rows that satisfy the condition
count = filtered_df.count()

# Show the count
print("Number of rows where optimalBid >= payingPrice:", count)

Number of rows where optimalBid >= payingPrice: 332129


In [61]:
from pyspark.sql.functions import col, sum as _sum

aggregated_df = filtered_df.agg(
    _sum('numClick').alias('total_numClick'),
    _sum('payingPrice').alias('total_payingPrice')
)

# Calculate ROI (numClick / payingPrice)
aggregated_df = aggregated_df.withColumn(
    'ROI', col('total_numClick') / (col('total_payingPrice') / 1000)
)

# Show the results
aggregated_df.show()

+--------------+-----------------+------------------+
|total_numClick|total_payingPrice|               ROI|
+--------------+-----------------+------------------+
|           129|          4110288|0.0313846620966706|
+--------------+-----------------+------------------+



### Find ROI under scenario 2: Budget = 1000 CNY

In [50]:
# UDF to compute the new column value
def calculate_value(t, alpha, positive_prob):
    return alpha * positive_prob * roi_dict.get(t)

calculate_value_udf = udf(calculate_value, DoubleType())

alpha = 0.4852294921875

test_df_with_prob = test_df_with_prob.withColumn('optimalBid', calculate_value_udf(col('hour'), lit(alpha), col('positive_prob')))

# Show the DataFrame with the new column
test_df_with_prob.show()

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|positive_prob|          optimalBid|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|       0|   216| 233|         1|177a166119fcf41b8...|  160|   600|         0| Monday|   0|         50|       26|   0.06616997|0.032107620813803806|
|       0|    94|  96|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|         14|      474|  0.066168584| 0.03210694837889605|
|       0|   216| 219|         2|14813226981bea2d8...|  300|   250|         5| Monday|   0|         24|     1806|   0.06616997|0.032107620813803806|
|       0|    80|  82|         1|1f1640b91b6c10733...|  300|   250|         0| Monday|   0|         53|   

In [52]:
# Filter rows where optimalBid >= payingPrice
filtered_df = test_df_with_prob.filter(col('optimalBid') >= col('payingPrice')/1000 )

# Count the number of rows that satisfy the condition
count = filtered_df.count()

# Show the count
print("Number of rows where optimalBid >= payingPrice:", count)

Number of rows where optimalBid >= payingPrice: 541809


In [53]:
filtered_df.show()

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|positive_prob|          optimalBid|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+--------------------+
|       0|    94|  96|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|         14|      474|  0.066168584| 0.03210694837889605|
|       0|   216| 219|         2|14813226981bea2d8...|  300|   250|         5| Monday|   0|         24|     1806|   0.06616997|0.032107620813803806|
|       0|   238| 239|         1|d8df91a3689429534...|  300|   250|         0| Monday|   0|         32|     3764|   0.06632049| 0.03218065592136554|
|       0|   275| 275|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|          5|   

In [54]:

aggregated_df = filtered_df.agg(
    _sum('numClick').alias('total_numClick'),
    _sum('payingPrice').alias('total_payingPrice')
)

# Calculate ROI (numClick / payingPrice)
aggregated_df = aggregated_df.withColumn(
    'ROI', col('total_numClick') / (col('total_payingPrice') / 1000)
)

# Show the results
aggregated_df.show()

+--------------+-----------------+-------------------+
|total_numClick|total_payingPrice|                ROI|
+--------------+-----------------+-------------------+
|           221|          9951972|0.02220665411839985|
+--------------+-----------------+-------------------+



### Find ROI under scenario 3: Budget = 1500 CNY

In [56]:

# UDF to compute the new column value
def calculate_value(t, alpha, positive_prob):
    return alpha * positive_prob * roi_dict.get(t)

calculate_value_udf = udf(calculate_value, DoubleType())

alpha = 0.604248046875

test_df_with_prob = test_df_with_prob.withColumn('optimalBid', calculate_value_udf(col('hour'), lit(alpha), col('positive_prob')))

# Show the DataFrame with the new column
test_df_with_prob.show()

+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+-------------------+
|numClick|region|city|adExchange|                 URL|width|height|floorPrice|weekday|hour|payingPrice|unique_id|positive_prob|         optimalBid|
+--------+------+----+----------+--------------------+-----+------+----------+-------+----+-----------+---------+-------------+-------------------+
|       0|   216| 233|         1|177a166119fcf41b8...|  160|   600|         0| Monday|   0|         50|       26|   0.06616997|0.03998307497568021|
|       0|    94|  96|         2|b9b27f86cd5409b50...|  468|    60|         5| Monday|   0|         14|      474|  0.066168584|0.03998223760390829|
|       0|   216| 219|         2|14813226981bea2d8...|  300|   250|         5| Monday|   0|         24|     1806|   0.06616997|0.03998307497568021|
|       0|    80|  82|         1|1f1640b91b6c10733...|  300|   250|         0| Monday|   0|         53|     2453

In [57]:
# Filter rows where optimalBid >= payingPrice
filtered_df = test_df_with_prob.filter(col('optimalBid') >= col('payingPrice')/1000 )

# Count the number of rows that satisfy the condition
count = filtered_df.count()

# Show the count
print("Number of rows where optimalBid >= payingPrice:", count)

Number of rows where optimalBid >= payingPrice: 649023


In [58]:

aggregated_df = filtered_df.agg(
    _sum('numClick').alias('total_numClick'),
    _sum('payingPrice').alias('total_payingPrice')
)

# Calculate ROI (numClick / payingPrice)
aggregated_df = aggregated_df.withColumn(
    'ROI', col('total_numClick') / (col('total_payingPrice') / 1000)
)

# Show the results
aggregated_df.show()

+--------------+-----------------+--------------------+
|total_numClick|total_payingPrice|                 ROI|
+--------------+-----------------+--------------------+
|           280|         14812002|0.018903589129950157|
+--------------+-----------------+--------------------+



### Areas for further exploration
* The ML model was trained and the bidding algorithm was found as if the entire impressions were shown in the same URL and from the same advertisers.  Ideally, would have different bidding models for each URL.

* Fine tune the ML model

* Fine-tune the pricing model to incorporate more temporal factors
 

In [44]:
# Count the number of unique values in the URL column
unique_url_count = test_df.select('URL').distinct().count()

# Show the count
print("Number of unique values in the URL column:", unique_url_count)

Number of unique values in the URL column: 751747


In [45]:
test_df.count()

2007563